<a href="https://colab.research.google.com/github/TheCiceerGuy/Simulation_Modeling-Labs/blob/main/Lab_3_Simulation_of_Single_Queue_Server_System_(M_M_1_Model).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***M/M/c System Implementation***

In [4]:

import simpy
import random

class QueueSystem:
    def __init__(self, env, num_servers, arrival_rate, service_rate):
        self.env = env
        self.servers = simpy.Resource(env, num_servers)
        self.arrival_rate = arrival_rate
        self.service_rate = service_rate
        self.total_wait_time = 0
        self.total_customers = 0
        self.server_busy_time = 0
        self.customer_log = []  # Stores arrival & service times

    def customer_process(self):
        customer_id = 1
        while True:
            yield self.env.timeout(random.expovariate(self.arrival_rate))  # Time until next customer arrives
            self.env.process(self.serve_customer(customer_id))
            customer_id += 1

    def serve_customer(self, customer_id):
        arrival_time = self.env.now
        with self.servers.request() as request:
            yield request  # Wait for a free server
            wait_time = self.env.now - arrival_time  # Calculate wait time
            self.total_wait_time += wait_time
            self.total_customers += 1

            service_time = random.expovariate(self.service_rate)  # Generate service time
            self.server_busy_time += service_time
            service_start_time = self.env.now
            yield self.env.timeout(service_time)  # Serve the customer
            service_end_time = self.env.now

            # Store customer log (arrival, service start, service end)
            self.customer_log.append((customer_id, arrival_time, service_start_time, service_end_time))

# Get user input
arrival_rate = float(input("Enter the customer arrival rate (λ): "))
service_rate = float(input("Enter the service rate per server (μ): "))
num_servers = int(input("Enter the number of servers (c): "))

# Set up simulation
env = simpy.Environment()
queue_system = QueueSystem(env, num_servers, arrival_rate, service_rate)
env.process(queue_system.customer_process())

# Run simulation for 100 time units
env.run(until=100)

# Calculate statistics
average_wait_time = queue_system.total_wait_time / queue_system.total_customers if queue_system.total_customers else 0
server_utilization = (queue_system.server_busy_time / (num_servers * 100)) * 100  # As percentage

# Display customer log
print("\nCustomer Log:")
print(f"{'ID':<5} {'Arrived':<10} {'Service Start':<15} {'Service End'}")
for customer in queue_system.customer_log:
    print(f"{customer[0]:<5} {customer[1]:<10.2f} {customer[2]:<15.2f} {customer[3]:.2f}")

# Display final results
print("\nSimulation Results:")
print(f"Total customers served: {queue_system.total_customers}")
print(f"Average waiting time: {average_wait_time:.2f} time units")
print(f"Server utilization: {server_utilization:.2f}%")




Enter the customer arrival rate (λ): 10
Enter the service rate per server (μ): 2
Enter the number of servers (c): 3

Customer Log:
ID    Arrived    Service Start   Service End
1     0.05       0.05            0.44
2     0.06       0.06            0.67
4     0.21       0.44            0.79
6     0.36       0.79            0.97
3     0.13       0.13            1.30
8     0.62       1.30            1.63
5     0.23       0.67            1.63
9     0.75       1.63            1.72
11    1.14       1.72            1.78
10    1.09       1.63            2.21
12    1.15       1.78            2.35
7     0.59       0.97            2.45
13    1.23       2.21            2.73
15    1.54       2.45            2.76
17    1.57       2.76            3.13
14    1.28       2.35            3.36
19    1.78       3.36            3.44
20    2.02       3.44            3.44
18    1.66       3.13            3.62
21    2.02       3.44            3.98
16    1.55       2.73            3.99
23    2.08       3.98     

***M/M/1 System Implementation***

In [7]:
import simpy
import random

class MM1Queue:
    def __init__(self, env, arrival_rate, service_rate):
        self.env = env
        self.server = simpy.Resource(env, capacity=1)  # M/M/1 has only 1 server
        self.arrival_rate = arrival_rate
        self.service_rate = service_rate
        self.total_wait_time = 0
        self.total_customers = 0
        self.server_busy_time = 0
        self.customer_log = []  # Stores arrival & service times

    def customer_process(self):
        customer_id = 1
        while True:
            yield self.env.timeout(random.expovariate(self.arrival_rate))  # Poisson arrival
            self.env.process(self.serve_customer(customer_id))
            customer_id += 1

    def serve_customer(self, customer_id):
        arrival_time = self.env.now
        with self.server.request() as request:
            yield request  # Wait for server to be free
            wait_time = self.env.now - arrival_time  # Time spent waiting
            self.total_wait_time += wait_time
            self.total_customers += 1

            service_time = random.expovariate(self.service_rate)  # Exponential service time
            self.server_busy_time += service_time
            service_start_time = self.env.now
            yield self.env.timeout(service_time)  # Processing time
            service_end_time = self.env.now

            # Store log data
            self.customer_log.append((customer_id, arrival_time, service_start_time, service_end_time))

# Get user input
arrival_rate = float(input("Enter the customer arrival rate (λ): "))
service_rate = float(input("Enter the service rate (μ): "))

# Set up simulation
env = simpy.Environment()
queue_system = MM1Queue(env, arrival_rate, service_rate)
env.process(queue_system.customer_process())

# Run simulation for 100 time units
env.run(until=100)

# Calculate statistics
average_wait_time = queue_system.total_wait_time / queue_system.total_customers if queue_system.total_customers else 0
server_utilization = (queue_system.server_busy_time / 100) * 100  # Since we have only 1 server

# Display customer log
print("\nCustomer Log:")
print(f"{'ID':<5} {'Arrived':<10} {'Service Start':<15} {'Service End'}")
for customer in queue_system.customer_log:
    print(f"{customer[0]:<5} {customer[1]:<10.2f} {customer[2]:<15.2f} {customer[3]:.2f}")

# Display final results
print("\nSimulation Results:")
print(f"Total customers served: {queue_system.total_customers}")
print(f"Average waiting time: {average_wait_time:.2f} time units")
print(f"Server utilization: {server_utilization:.2f}%")


Enter the customer arrival rate (λ): 5
Enter the service rate (μ): 6

Customer Log:
ID    Arrived    Service Start   Service End
1     0.01       0.01            0.21
2     0.24       0.24            0.35
3     0.47       0.47            0.96
4     0.53       0.96            1.21
5     0.61       1.21            1.21
6     0.83       1.21            1.34
7     1.42       1.42            1.54
8     1.43       1.54            1.88
9     1.43       1.88            2.01
10    1.72       2.01            2.30
11    1.87       2.30            2.64
12    2.15       2.64            2.69
13    2.36       2.69            3.33
14    2.41       3.33            4.10
15    2.57       4.10            4.31
16    2.62       4.31            4.62
17    2.85       4.62            4.70
18    2.88       4.70            4.72
19    2.91       4.72            4.84
20    2.94       4.84            5.02
21    2.99       5.02            5.09
22    3.10       5.09            5.15
23    3.18       5.15            5.